# Named Entity Recognition Model
In this python notebook, a NER model will be created by using the data scraped from LinkedIn jobs.
The steps are:
1. Data Cleaning
2. Data Annotating
3. Data Preparation
4. Data Modelling
5. Evaluation
6. Predict on Test Data
7. Visualize data using Tableau

## Imports

In [2]:
%reload_ext autoreload
%autoreload 2
import os
import sys
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
sys.path.append(os.getenv("PROJECT_DIR"))
import pandas as pd
import re
import spacy
import numpy as np
import json
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from tqdm import tqdm
from src.data import DataLoader
from src.data import DataPreprocessor

## 1. Data Preprocessing

In [3]:
# Load data
df = DataLoader.load_data("../data/raw/linkedin_jobs_train.csv")
df = df.drop_duplicates().reset_index(drop=True)

# Preprocess Job Descriptions
texts = []
data_preprocessor = DataPreprocessor(model_name="en_core_web_sm")
for i, row in tqdm(df.iterrows()):
    texts.append(data_preprocessor.preprocess_job_desc(row['job_description'])+"\n")

# Save Jobs in form of jobs.txt
with open("../data/interim/jobs_train.txt", "w") as f:
    f.writelines(texts)

214it [00:03, 67.81it/s]


## 2. Data Annotating

Annotate data using this link https://tecoholic.github.io/ner-annotator/

## 3. Data Preparation 

Split data for train set, development set and test set.

In [4]:
# Prepare Train and Dev
# Open the annotations
with open("../data/interim/annotations.json", "r") as f:
    annotations_json = json.load(f)

# Set params for random select data
total_size = len(annotations_json['annotations'])
train_len, dev_len, test_len = int(total_size * 0.7), int(total_size * 0.15), int(total_size * 0.15) 
data = []
for aj in annotations_json['annotations']:
    data.append((aj[0], aj[1]))    

# Randomized the data
np.random.shuffle(data)

# Prepare Train Data
train_data = data[:train_len] 
db = DataPreprocessor.convert_to_doc_bin(train_data)
db.to_disk("../data/processed/train.spacy")

# Prepare Dev Data
dev_data = data[train_len : train_len + dev_len] 
db = DataPreprocessor.convert_to_doc_bin(dev_data)
db.to_disk("../data/processed/dev.spacy")

# Prepare Test Data
test_data = data[train_len + dev_len:] 
for data in test_data:
    texts.append(data[0])

with open("../data/interim/jobs_test.txt", "w") as f:
    f.writelines(texts)

## 4. Data Modelling (From Scratch)
Steps
1. Download base configuration from spacy website according to NER modelling needs, create configuration for our training.
2. Train the model using the configuration along with the train data and test data.

In [ ]:
# 1
!python -m spacy init fill-config ../config/base_config.cfg ../config/config.cfg
# 2
!python -m spacy train ../config/config.cfg --output ../models --paths.train ../data/processed/train.spacy --paths.dev ../data/processed/dev.spacy

## 5. Model Evaluation
Pick the best model

![image.png](../reports/figures/evaluation.png)

## 6. Predict on Test Data

In [ ]:
# Load the Test Data
with open("../data/interim/jobs_test.txt", "r") as f:
    test_data = f.readlines()

# Access the Test Data and load Spacy Model
best_model = spacy.load("../models/model-best")
i = 0
hard_skills = []
soft_skills = []
for text in test_data:
    docs = best_model(text)
    for doc in docs.ents:
        text = doc.text
        text = re.sub(" - ", "-", text)
        text = re.sub(" / ", "/", text)
        text = re.sub(" ", "_", text)
        if doc.label_ == "HARD SKILL":
            hard_skills.append(text)
        elif doc.label_ == "SOFT SKILL":
            text = re.sub("-", "_", text)
            soft_skills.append(text)

### Word Cloud 

Hard Skills

In [ ]:
# join the list and lowercase all the words
text = ' '.join(hard_skills).lower()

#create the wordcloud object
wordcloud = WordCloud().generate(text)

#plot the wordcloud object
plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

Count Plot

In [ ]:
hard_skill_df = pd.DataFrame({
    "hard_skill": hard_skills,
})
hard_skill_df['hard_skill'].value_counts().sort_values(ascending=True)[-30:].plot(kind="barh", figsize=(10, 7), title="Top 30 Data Scientist Hard Skills")
plt.show()

Soft Skills

In [ ]:
# join the list and lowercase all the words
text = ' '.join(soft_skills).lower()

#create the wordcloud object
wordcloud = WordCloud().generate(text)

#plot the wordcloud object
plt.imshow(wordcloud, interpolation='bilInear')
plt.axis('off')
plt.show()

Count Plot

In [ ]:
soft_skill_df = pd.DataFrame({
    "soft_skill": soft_skills,
})
soft_skill_df['soft_skill'].value_counts().sort_values(ascending=True)[-30:].plot(kind="barh", figsize=(10, 7), title="Top 30 Data Scientist Soft Skills")
plt.show()

In [ ]:
# save data
hard_skill_df.to_csv("../reports/hard_skill.csv", index=False)
soft_skill_df.to_csv("../reports/soft_skill.csv", index=False)

## 7. Visualize in Tableau
Link: https://public.tableau.com/views/DataScienceJobSkillsPlatform/Frontpage?:language=en-US&:display_count=n&:origin=viz_share_link